## 뉴스 크롤링 최종본

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
import os
import pandas as pd
import urllib
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
import math
from urllib.request import HTTPError
from urllib.request import URLError

In [2]:
def date_range(start, end):
    date_index = pd.date_range(start=start, end=end,freq='D')
    # type(dt_index) => DatetimeIndex
    # DatetimeIndex => list(str)
    date_list = date_index.strftime("%Y-%m-%d").tolist()
    return date_list

In [3]:
date_list = date_range('20170101','20170101')

In [15]:
links_dt_list = []
links_dd_list = []
links_list = []
links_list_tmp = []
title_dt_list=[]
title_dd_list=[]
title_list = []
title_list_tmp = []
cont_list = []
news_date_list = []
cont_kor_list = []
headers = { "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36" }
q = '삼성중공업'
try:
    for date in tqdm(date_list):
        time.sleep(1)
        print('추출한 날짜',date)
        page = 1
        url = 'https://finance.naver.com/news/news_search.naver?rcdate=1&q={q}&x=0&y=0&sm=all.basic&pd=4&' + f'stDateStart={date}&stDateEnd={date}'
        q_enc = urllib.parse.quote_plus(q, encoding='euc-kr')
        url.format(q=q_enc)
        res = requests.get(url.format(q=q_enc),headers = headers)
        soup = BeautifulSoup(res.text, 'lxml')
        strong = soup.find_all('strong')
        article_num = strong[-3] # 날짜와 이름 검색 후 네이버에서 제공하는 기사 갯수를 저장 합니다 (몇개의 기사가 있는지 확인용)
        article_num = int(re.sub(r"[^a-zA-Z0-9]","",article_num.text))
        page_num = math.ceil(article_num/20) + 1
        print(page_num-1)
        for page in range(1,page_num) :
            url = 'https://finance.naver.com/news/news_search.naver?rcdate=1&q={q}&x=0&y=0&sm=all.basic&pd=4&' + f'stDateStart={date}&stDateEnd={date}&page={page}'
            q_enc = urllib.parse.quote_plus(q, encoding='euc-kr')
            url.format(q=q_enc)
            res = requests.get(url.format(q=q_enc),headers=headers)
            soup = BeautifulSoup(res.text, 'lxml')
            elem_news = soup.select_one('div.newsSchResult dl.newsList')
            news_subject_dt = elem_news.find_all('dt',{'class' : 'articleSubject'})
            news_subject_dd = elem_news.find_all('dd',{'class' : 'articleSubject'})
            for dt in news_subject_dt:
                href_dt = dt.find("a")["href"]
                links_dt = 'https://finance.naver.com/' + href_dt
                links_dt_list.append(links_dt)
                title_dt = dt.text[1:-1]
                title_dt_list.append(title_dt)
            for dd in news_subject_dd:
                href_dd = dd.find("a")["href"]
                links_dd = 'https://finance.naver.com/' + href_dd
                links_dd_list.append(links_dd)
                title_dd = dd.text[1:-1]
                title_dd_list.append(title_dd)
                
            links_list_tmp = links_dt_list + links_dd_list
            links_list.extend(links_list_tmp)
            links_dt_list.clear()
            links_dd_list.clear()
            links_list_tmp.clear()
            
            title_list_tmp = title_dt_list + title_dd_list
            title_list.extend(title_list_tmp)
            title_dt_list.clear()
            title_dd_list.clear()
            title_list_tmp.clear()
            
    for url in tqdm(links_list):
        time.sleep(1)
        res = requests.get(url,headers=headers)
        soup = BeautifulSoup(res.text, 'lxml')
        news_date = soup.find('span', {'class' : 'article_date'})
        contents = soup.find('div',{'id' : 'content'})
        news_date_list.append(news_date.get_text())
        if contents.find_all('p') != None:
            p_tags = contents.find_all('p')
            
            for i in range(len(p_tags)):
                contents.find('p').extract()
            if (soup.find('div',{'class' : 'link_news'})) != None :
                link_news = soup.find('div',{'class' : 'link_news'}).get_text(strip=True)
            
                cont_list.append(contents.get_text(strip=True).replace(link_news,''))
            else :
                cont_list.append(contents.get_text(strip=True))
        else:
            
            if (soup.find('div',{'class' : 'link_news'})) != None :
                link_news = soup.find('div',{'class' : 'link_news'}).get_text(strip=True)
            
                cont_list.append(contents.get_text(strip=True).replace(link_news,''))
            else :
                cont_list.append(contents.get_text(strip=True))
                                     
except (HTTPError, URLError, TypeError, AttributeError) as e :
    print(e)
         
else :
    print('크롤링 완료')

for i in range(len(cont_list)):
    cont_list[i] = str(cont_list[i])
    content_kor_tmp = re.compile('[가-힣]+').findall(cont_list[i])
    content_kor = ' '.join(content_kor_tmp)
    cont_kor_list.append(content_kor)
    
print(np.unique(len(cont_list)))
print(np.unique(len(cont_kor_list)))

  0%|                                                                        | 0/1 [00:00<?, ?it/s]

추출한 날짜 2017-01-01
1


100%|████████████████████████████████████████████████████████████████| 5/5 [00:06<00:00,  1.28s/it]

크롤링 완료
[5]
[5]


In [2]:
# tmpaaa_list = []
# for i in range(len(cont_list)):
#     cont_list[i] = str(cont_list[i])
#     aaa = re.compile('[가-힣]+').findall(cont_list[i])
#     aaa_kor = ' '.join(aaa)
#     tmpaaa_list.append(aaa_kor)
# print(tmpaaa_list[0])
# print('-------------')
# print('-------------')
# print(cont_kor_list[0])

In [9]:
len(links_list),len(news_date_list),len(title_list),len(cont_list)

(5, 5, 5, 5)

In [10]:
data={'link' : links_list, 'date' : news_date_list , 'title' : title_list, 'contents' : cont_list}
df_content = pd.DataFrame(
      data=data
)
data_kor={'link' : links_list, 'date' : news_date_list , 'title' : title_list, 'contents' : cont_kor_list}
df_content_kor = pd.DataFrame(
      data=data_kor
)

In [11]:
df_content.isnull().sum(), df_content_kor.isnull().sum()

(link        0
 date        0
 title       0
 contents    0
 dtype: int64,
 link        0
 date        0
 title       0
 contents    0
 dtype: int64)

In [12]:
df_content

,link,date,title,contents
0,https://finance.naver.com//news/news_read.nave...,2017-01-01 21:38,스러진 '마산의 골리앗'…불 꺼진 조선소 도시,"<앵커>지금 보신 것처럼 외교와 안보가 걱정이지만, 경제도 만만치 않습니다. 우리 ..."
1,https://finance.naver.com//news/news_read.nave...,2017-01-01 17:06,수주절벽 지속 `조선 빅3` 올 목표 수립 난항,"작년 총수주액, 목표치의 17%업계, 시장 불확실성 확산 속목표치 제시 부담 분위기..."
2,https://finance.naver.com//news/news_read.nave...,2017-01-01 14:29,"[선박톡톡]엘리자베스, 인디펜던스, 을지문덕…배 이름에 담긴 의미는?","[편집자 주] 선박. 물에 떠서 사람·가축·물자를 싣고, 물 위로 이동할 수 있는 ..."
3,https://finance.naver.com//news/news_read.nave...,2017-01-01 07:00,조선 불황 '거제상품권' 판매 뚝…시 소비진작 나선다,(거제=연합뉴스) 이경욱 기자 = 조선 불황이 강타한 경남 거제시의 '거제사랑상품권...
4,https://finance.naver.com//news/news_read.nave...,2017-01-01 01:51,"겨냥하고 쏘면 늦는다, 일단 쏘고 겨냥하라",42%. LG경제연구원에 따르면 1995년 매출액 기준 미국 제조업 500대 기업(...


In [13]:
df_content_kor

,link,date,title,contents
0,https://finance.naver.com//news/news_read.nave...,2017-01-01 21:38,스러진 '마산의 골리앗'…불 꺼진 조선소 도시,앵커 지금 보신 것처럼 외교와 안보가 걱정이지만 경제도 만만치 않습니다 우리 경제의...
1,https://finance.naver.com//news/news_read.nave...,2017-01-01 17:06,수주절벽 지속 `조선 빅3` 올 목표 수립 난항,작년 총수주액 목표치의 업계 시장 불확실성 확산 속목표치 제시 부담 분위기현대중공업...
2,https://finance.naver.com//news/news_read.nave...,2017-01-01 14:29,"[선박톡톡]엘리자베스, 인디펜던스, 을지문덕…배 이름에 담긴 의미는?",편집자 주 선박 물에 떠서 사람 가축 물자를 싣고 물 위로 이동할 수 있는 구조물을...
3,https://finance.naver.com//news/news_read.nave...,2017-01-01 07:00,조선 불황 '거제상품권' 판매 뚝…시 소비진작 나선다,거제 연합뉴스 이경욱 기자 조선 불황이 강타한 경남 거제시의 거제사랑상품권 판매량이...
4,https://finance.naver.com//news/news_read.nave...,2017-01-01 01:51,"겨냥하고 쏘면 늦는다, 일단 쏘고 겨냥하라",경제연구원에 따르면 년 매출액 기준 미국 제조업 대 기업 포춘 선정 중 년 후인 년...


In [ ]:
df_content.to_csv('df_content.csv',encoding='utf-8-sig')

In [ ]:
df_content_kor.to_csv('df_content.csv',encoding='utf-8-sig')